In [51]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [52]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForObjectDetection

processor = AutoImageProcessor.from_pretrained("microsoft/table-transformer-structure-recognition-v1.1-all")
model = AutoModelForObjectDetection.from_pretrained("microsoft/table-transformer-structure-recognition-v1.1-all")

In [53]:
model

TableTransformerForObjectDetection(
  (model): TableTransformerModel(
    (backbone): TableTransformerConvModel(
      (conv_encoder): TableTransformerConvEncoder(
        (model): ResNetBackbone(
          (embedder): ResNetEmbeddings(
            (embedder): ResNetConvLayer(
              (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
              (normalization): TableTransformerFrozenBatchNorm2d()
              (activation): ReLU()
            )
            (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          )
          (encoder): ResNetEncoder(
            (stages): ModuleList(
              (0): ResNetStage(
                (layers): Sequential(
                  (0): ResNetBasicLayer(
                    (shortcut): Identity()
                    (layer): Sequential(
                      (0): ResNetConvLayer(
                        (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(

In [54]:
# from PIL import Image
# import torch
# from transformers import AutoImageProcessor, AutoModelForObjectDetection


# # 2. Tải ảnh bảng đã cắt của bạn
# # Thay thế bằng đường dẫn đến ảnh bảng của bạn
# cropped_table_image_path = "/content/Screenshot from 2025-05-25 11-11-13.png"
# try:
#     image = Image.open(cropped_table_image_path).convert("RGB")
# except FileNotFoundError:
#     print(f"Lỗi: Không tìm thấy tệp ảnh tại: {cropped_table_image_path}")
#     exit()
# width, height = image.size
# image.resize((int(width*0.5),int(height*0.5)))





In [55]:
# from transformers import DetrFeatureExtractor
# feature_extractor = DetrFeatureExtractor()

In [56]:
# encoding = feature_extractor(image,return_tensors="pt")
# encoding.keys()

In [57]:
# with torch.no_grad():
#   outputs = model(**encoding)

In [58]:
# import matplotlib.pyplot as plt

# # colors for visualization
# COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
#           [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# def plot_results(pil_img, scores, labels, boxes):
#     plt.figure(figsize=(25,20))
#     plt.imshow(pil_img)
#     ax = plt.gca()
#     colors = COLORS * 100
#     for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
#         ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
#                                    fill=False, color=c, linewidth=3))
#         text = f'{model.config.id2label[label]}: {score:0.2f}'
#         ax.text(xmin, ymin, text, fontsize=15,
#                 bbox=dict(facecolor='yellow', alpha=0.5))
#     plt.axis('off')
#     plt.show()

In [59]:
# target_sizes = [image.size[::-1]]
# results = feature_extractor.post_process_object_detection(outputs, threshold=0.6, target_sizes=target_sizes)[0]
# plot_results(image, results['scores'], results['labels'], results['boxes'])

In [60]:
# model.config.id2label

In [61]:
# results

In [62]:
# Cài đặt Poppler
!apt-get update
!apt-get install -y poppler-utils

# (Tùy chọn) Cài đặt các gói Python có thể cần Poppler nếu bạn sử dụng chúng
# Ví dụ: nếu bạn định dùng pdf2image
# !pip install pdf2image
!pip install loguru pdf2image

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
Defaulting to user installation because normal site-packages is not writeable


In [63]:
PDF_NAME = '0af43d695e0fa436b1c1dfc150d48182.pdf'

In [64]:
import os
import glob

# Define the directory containing the images
images_dir = '/home/tiamo/WDM-AI-TEMIS/data-finetune/images'

# Remove all files in the images directory
files = glob.glob(os.path.join(images_dir, '*'))
for f in files:
    os.remove(f)
    logger.info(f'Removed file {f}')

from pathlib import Path
from loguru import logger
from pdf2image import convert_from_path
from collections import Counter

def get_images_from_pdf(pdf_path, max_pages=None,dpi_resolution=144,
                        save_dir='/tmp',save_images=False,save_type='png'):
    """
    Convert PDF pages to images
    """
    pdf_path = Path(pdf_path)
    assert pdf_path.exists(), f'PDF file {pdf_path} does not exist'

    pdf_fname = pdf_path.name

    images = convert_from_path(pdf_path,dpi=dpi_resolution)

    # PIL.PpmImagePlugin.PpmImageFile  -> PIL.Image.Image
    images = [img.convert('RGB') for img in images]

    # Resizing to the most common image size so that we can stack in pytorch tensor

    # 1) find the most common image size
    img_size_counter = Counter()
    for img in images:
        img_size_counter[img.size] += 1
    common_img_size, common_img_size_count = img_size_counter.most_common(1)[0]

    # 2) if pages are not the same size, resize to the most common image size
    if len(images) != common_img_size_count:
        logger.info(f'Resizing {len(images)} images to {common_img_size}')
        images = [img.resize(common_img_size) for img in images]

    # 3) save images
    if save_images:
        save_dir = Path(save_dir)
        save_dir.mkdir(parents=True,exist_ok=True)

        for page_idx, img in enumerate(images):
            img_path = save_dir / f'{pdf_fname}_{page_idx+1}.{save_type}'
            if not img_path.exists():
                img.save(img_path)
                logger.info(f'Saved image {img_path}')
            else:
                logger.info(f'Image {img_path} already exists')

    return images

if __name__ == '__main__':
    get_images_from_pdf(pdf_path=f'/home/tiamo/WDM-AI-TEMIS/data-finetune/pdf4tabel/{PDF_NAME}',save_dir='/home/tiamo/WDM-AI-TEMIS/data-finetune/images',save_images=True)


2025-05-30 11:38:36.651 | INFO     | __main__:<module>:11 - Removed file /home/tiamo/WDM-AI-TEMIS/data-finetune/images/0af43d695e0fa436b1c1dfc150d48182.pdf_7.png
2025-05-30 11:38:36.652 | INFO     | __main__:<module>:11 - Removed file /home/tiamo/WDM-AI-TEMIS/data-finetune/images/0af43d695e0fa436b1c1dfc150d48182.pdf_3.png
2025-05-30 11:38:36.653 | INFO     | __main__:<module>:11 - Removed file /home/tiamo/WDM-AI-TEMIS/data-finetune/images/0af43d695e0fa436b1c1dfc150d48182.pdf_4.png
2025-05-30 11:38:36.655 | INFO     | __main__:<module>:11 - Removed file /home/tiamo/WDM-AI-TEMIS/data-finetune/images/0af43d695e0fa436b1c1dfc150d48182.pdf_8.png
2025-05-30 11:38:36.658 | INFO     | __main__:<module>:11 - Removed file /home/tiamo/WDM-AI-TEMIS/data-finetune/images/0af43d695e0fa436b1c1dfc150d48182.pdf_1.png
2025-05-30 11:38:36.660 | INFO     | __main__:<module>:11 - Removed file /home/tiamo/WDM-AI-TEMIS/data-finetune/images/0af43d695e0fa436b1c1dfc150d48182.pdf_6.png
2025-05-30 11:38:36.662 | IN

# Full pipeline

In [65]:
# !pip install -q easyocr

## Load model

In [66]:
from transformers import AutoModelForObjectDetection

model = AutoModelForObjectDetection.from_pretrained("microsoft/table-transformer-detection", revision="no_timm")

config.json:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

In [67]:
model.config.id2label

{0: 'table', 1: 'table rotated'}

In [68]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print("")

## Load image

In [84]:
from PIL import Image
from huggingface_hub import hf_hub_download
PAGE_NUMBER = 3

# let's load an example image
file_path = f'/home/tiamo/WDM-AI-TEMIS/data-finetune/images/{PDF_NAME}_{PAGE_NUMBER}.png'
image = Image.open(file_path).convert("RGB")
# let's display it a bit smaller
width, height = image.size
# display(image.resize((int(0.6*width), (int(0.6*height)))))

## Prepare image for model

In [85]:
from torchvision import transforms

class MaxResize(object):
    def __init__(self, max_size=800):
        self.max_size = max_size

    def __call__(self, image):
        width, height = image.size
        current_max_size = max(width, height)
        scale = self.max_size / current_max_size
        resized_image = image.resize((int(round(scale*width)), int(round(scale*height))))

        return resized_image

detection_transform = transforms.Compose([
    MaxResize(800),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [86]:
pixel_values = detection_transform(image).unsqueeze(0)
pixel_values = pixel_values.to(device)
print(pixel_values.shape)

torch.Size([1, 3, 800, 618])


## Forward pass

In [87]:
import torch

with torch.no_grad():
  outputs = model(pixel_values)

In [88]:
outputs.logits.shape

torch.Size([1, 15, 3])

## Post processing

In [89]:
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)


def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b


# update id2label to include "no object"
id2label = model.config.id2label
id2label[len(model.config.id2label)] = "no object"


def outputs_to_objects(outputs, img_size, id2label):
    m = outputs.logits.softmax(-1).max(-1)
    pred_labels = list(m.indices.detach().cpu().numpy())[0]
    pred_scores = list(m.values.detach().cpu().numpy())[0]
    pred_bboxes = outputs['pred_boxes'].detach().cpu()[0]
    pred_bboxes = [elem.tolist() for elem in rescale_bboxes(pred_bboxes, img_size)]

    objects = []
    for label, score, bbox in zip(pred_labels, pred_scores, pred_bboxes):
        class_label = id2label[int(label)]
        if not class_label == 'no object':
            objects.append({'label': class_label, 'score': float(score),
                            'bbox': [float(elem) for elem in bbox]})

    return objects

In [90]:
objects = outputs_to_objects(outputs, image.size, id2label)

In [91]:
print(objects)

[{'label': 'table', 'score': 0.9825701117515564, 'bbox': [50.8600959777832, 16.32668685913086, 1088.55419921875, 1552.1962890625]}]


## Visualize

In [92]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Patch

def fig2img(fig):
    """Convert a Matplotlib figure to a PIL Image and return it"""
    import io
    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    img = Image.open(buf)
    return img


def visualize_detected_tables(img, det_tables, out_path=None):
    plt.imshow(img, interpolation="lanczos")
    fig = plt.gcf()
    fig.set_size_inches(20, 20)
    ax = plt.gca()

    for det_table in det_tables:
        bbox = det_table['bbox']

        if det_table['label'] == 'table':
            facecolor = (1, 0, 0.45)
            edgecolor = (1, 0, 0.45)
            alpha = 0.3
            linewidth = 2
            hatch='//////'
        elif det_table['label'] == 'table rotated':
            facecolor = (0.95, 0.6, 0.1)
            edgecolor = (0.95, 0.6, 0.1)
            alpha = 0.3
            linewidth = 2
            hatch='//////'
        else:
            continue

        rect = patches.Rectangle(bbox[:2], bbox[2]-bbox[0], bbox[3]-bbox[1], linewidth=linewidth,
                                    edgecolor='none',facecolor=facecolor, alpha=0.1)
        ax.add_patch(rect)
        rect = patches.Rectangle(bbox[:2], bbox[2]-bbox[0], bbox[3]-bbox[1], linewidth=linewidth,
                                    edgecolor=edgecolor,facecolor='none',linestyle='-', alpha=alpha)
        ax.add_patch(rect)
        rect = patches.Rectangle(bbox[:2], bbox[2]-bbox[0], bbox[3]-bbox[1], linewidth=0,
                                    edgecolor=edgecolor,facecolor='none',linestyle='-', hatch=hatch, alpha=0.2)
        ax.add_patch(rect)

    plt.xticks([], [])
    plt.yticks([], [])

    legend_elements = [Patch(facecolor=(1, 0, 0.45), edgecolor=(1, 0, 0.45),
                                label='Table', hatch='//////', alpha=0.3),
                        Patch(facecolor=(0.95, 0.6, 0.1), edgecolor=(0.95, 0.6, 0.1),
                                label='Table (rotated)', hatch='//////', alpha=0.3)]
    plt.legend(handles=legend_elements, bbox_to_anchor=(0.5, -0.02), loc='upper center', borderaxespad=0,
                    fontsize=10, ncol=2)
    plt.gcf().set_size_inches(10, 10)
    plt.axis('off')

    if out_path is not None:
      plt.savefig(out_path, bbox_inches='tight', dpi=150)

    return fig

In [99]:
# fig = visualize_detected_tables(image, objects)

In [94]:
visualized_image = fig2img(fig)

## Crop table

In [95]:
def objects_to_crops(img, tokens, objects, class_thresholds, padding=10):
    """
    Process the bounding boxes produced by the table detection model into
    cropped table images and cropped tokens.
    """

    table_crops = []
    for obj in objects:
        if obj['score'] < class_thresholds[obj['label']]:
            continue

        cropped_table = {}

        bbox = obj['bbox']
        bbox = [bbox[0]-padding, bbox[1]-padding, bbox[2]+padding, bbox[3]+padding]

        cropped_img = img.crop(bbox)

        table_tokens = [token for token in tokens if iob(token['bbox'], bbox) >= 0.5]
        for token in table_tokens:
            token['bbox'] = [token['bbox'][0]-bbox[0],
                             token['bbox'][1]-bbox[1],
                             token['bbox'][2]-bbox[0],
                             token['bbox'][3]-bbox[1]]

        # If table is predicted to be rotated, rotate cropped image and tokens/words:
        if obj['label'] == 'table rotated':
            cropped_img = cropped_img.rotate(270, expand=True)
            for token in table_tokens:
                bbox = token['bbox']
                bbox = [cropped_img.size[0]-bbox[3]-1,
                        bbox[0],
                        cropped_img.size[0]-bbox[1]-1,
                        bbox[2]]
                token['bbox'] = bbox

        cropped_table['image'] = cropped_img
        cropped_table['tokens'] = table_tokens

        table_crops.append(cropped_table)

    return table_crops

In [96]:
objects

[{'label': 'table',
  'score': 0.9825701117515564,
  'bbox': [50.8600959777832,
   16.32668685913086,
   1088.55419921875,
   1552.1962890625]}]

In [100]:
tokens = []
detection_class_thresholds = {
    "table": 0.5,
    "table rotated": 0.7,
    "no object": 10,
}
crop_padding = 10

tables_crops = objects_to_crops(image, tokens, objects, detection_class_thresholds, padding=25) # Adjust padding higher to avoid miss information
cropped_table = tables_crops[0]['image'].convert("RGB")
# cropped_table

In [ ]:
import uuid

new_uuid = str(uuid.uuid4())
print(f'{new_uuid}.png')
cropped_table.save(f"/home/tiamo/WDM-AI-TEMIS/data-finetune/cropeed_table/{new_uuid}.png")


87b65cc0-022c-4467-b5ec-30172b6e6cf3.png
